In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/Shareddrives/SWC_Capstone/5월'

/content/drive/Shareddrives/SWC_Capstone/5월


# 01. 원하는 기간 내의 데이터로 자르기
- (non-mentaldisorder 제외)

In [ ]:
import pandas as pd

In [ ]:
bipolar = pd.read_csv("./psy_data/bipolar_with_diagnose.csv")
print("bipolar: ", len(bipolar))
depression = pd.read_csv("./psy_data/depress_with_diagnose.csv")
print("depression: ", len(depression))
panicdisorder = pd.read_csv("./psy_data/panicdisorder_with_diagnose.csv")
print("panicdisorder: ", len(panicdisorder))
panicattack = pd.read_csv("./psy_data/panicattack_with_diagnose.csv")
print("panicattack: ", len(panicattack))

bipolar:  14118
depression:  32581
panicdisorder:  203
panicattack:  643


In [ ]:
panic = pd.concat([panicdisorder, panicattack])

print("bipolar: ", len(bipolar))
print("depression: ", len(depression))
print("panic : ", len(panic))

bipolar:  14118
depression:  32581
panic :  846


In [ ]:
bipolar = bipolar[bipolar['date']<'2022-03-01 00:00:00']
depression = depression[depression['date']<'2022-03-01 00:00:00']
panic = panic[panic['date']<'2022-03-01 00:00:00']

print("bipolar: ", len(bipolar))
print("depression: ", len(depression))
print("panic : ", len(panic))

bipolar:  13313
depression:  31691
panic :  785


In [ ]:
depression.head()

,subreddit,author,title,text_context,date
890,depress,justthisoncebal,My girlfriend was diagnosed with depression. H...,"So, she's been having issues ever since I've m...",2022-02-28 22:35:23
891,depress,Ico_Krow,It looks like my thoughts are in real life but...,\n\nI have depression for 3 years and suicidal...,2022-02-28 21:41:47
892,depress,theatricalis,I just can’t do this anymore.,I was diagnosed with depression while I was in...,2022-02-28 21:27:32
893,depress,Honest-Worker-480,schools sucks when having depression,I’m one of those people that honestly you woul...,2022-02-28 21:26:27
894,depress,Heartlessmf,do I tell my friends I've been diagnosed with ...,[removed],2022-02-28 20:50:01


# 02. null 존재 행 제거

In [ ]:
bipolar = bipolar.drop(bipolar[bipolar['author'] == '[deleted]'].index)
bipolar = bipolar.drop(bipolar[bipolar['text_context'] == '[deleted]'].index)
bipolar = bipolar.drop(bipolar[bipolar['text_context'] == '[removed]'].index)
bipolar = bipolar.drop(bipolar[bipolar['text_context'].isnull()].index)

depression = depression.drop(depression[depression['author'] == '[deleted]'].index)
depression = depression.drop(depression[depression['text_context'] == '[deleted]'].index)
depression = depression.drop(depression[depression['text_context'] == '[removed]'].index)
depression = depression.drop(depression[depression['text_context'].isnull()].index)

panic = panic.drop(panic[panic['author'] == '[deleted]'].index)
panic = panic.drop(panic[panic['text_context'] == '[deleted]'].index)
panic = panic.drop(panic[panic['text_context'] == '[removed]'].index)
panic = panic.drop(panic[panic['text_context'].isnull()].index)

print("bipolar: ", len(bipolar))
print("depression: ", len(depression))
print("panic : ", len(panic))

bipolar:  12737
depression:  31226
panic :  760


# 03. 관련 게시글 10번 이상 작성한 사람의 게시글만 뽑음

In [ ]:
bipolar_grouped = bipolar.groupby(by=['author'], as_index=False).count()
depression_grouped = depression.groupby(by=['author'], as_index=False).count()
panic_grouped = panic.groupby(by=['author'], as_index=False).count()

bipolar_ten = bipolar_grouped[bipolar_grouped['text_context'] > 9]
depression_ten = depression_grouped[depression_grouped['text_context'] > 9]
panic_ten = panic_grouped[panic_grouped['text_context'] > 9]

In [ ]:
# 타이틀 같은 게시글이 null --> 공란으로 변경 ("")
bipolar = pd.merge(bipolar, bipolar_ten['author'], how='inner', on='author')
depression = pd.merge(depression, depression_ten['author'], how='inner', on='author')
panic = pd.merge(panic, panic_ten['author'], how='inner', on='author')
#결과 ==> bipolar:  240 | depression:  20 | panic :  0 (너무 적어서 포기)

# 04. 한번에 처리 (추후 추가적 데이터셋 고려)

In [ ]:
def preprocess_df(df):
    DF = df[df['date'] < '2022-03-01 00:00:00']
    # print(len(df))
    DF = DF.drop(DF[DF['author'] == '[deleted]'].index)
    DF = DF.drop(DF[DF['text_context'] == '[deleted]'].index)
    DF = DF.drop(DF[DF['text_context'] == '[removed]'].index)
    DF = DF.drop(DF[DF['text_context'].isnull()].index)
    DF.reset_index(inplace=True)
    # print(len(df))
    return DF

In [ ]:
def make_year_month_col(df, save_path):
    DF = preprocess_df(df)
    # print(DF)
    year = []
    month = []
    for index in range(len(DF)):
        date = DF['date'][index]
        timestamp = dt.strptime(date,'%Y-%m-%d %H:%M:%S')
        DF['date'][index] = timestamp
        year.append(timestamp.year)
        month.append(timestamp.month)
    
    DF['year'] = year
    DF['month'] = month

    DF.to_csv(save_path, mode='w', index = False)


In [ ]:
covid19 = pd.read_csv("./non_mental_data/covid19_origin.csv")
print("covid19: ", len(covid19))
relationship = pd.read_csv("./non_mental_data/realationship_origin.csv")
print("relationship: ", len(relationship))
teaching = pd.read_csv("./non_mental_data/teaching_origin.csv")
print("teaching: ", len(teaching))

In [ ]:
preprocess_df(covid19)
preprocess_df(relationship)
preprocess_df(teaching)

# 05. 전처리 결과 저장

In [ ]:
## 데이터 불러오기

## 정신질환 데이터
depress_df = pd.read_csv("./psy_data/depress_with_diagnose.csv", sep = ',')
bipolar_df = pd.read_csv("./psy_data/bipolar_with_diagnose.csv", sep = ',')
panic_dis_df = pd.read_csv("./psy_data/panicdisorder_with_diagnose.csv", sep = ',')
panic_att_df = pd.read_csv("./psy_data/panicattack_with_diagnose.csv", sep = ',')
panic_df = pd.concat([panic_dis_df,panic_att_df])

## 일반 데이터
covid_df = pd.read_csv("./non_mental_data/covid19_origin.csv", sep = ',')
relationship_df = pd.read_csv("./non_mental_data/realationship_origin.csv", sep = ',')
teaching_df = pd.read_csv("./non_mental_data/teaching_origin.csv", sep = ',')

In [ ]:
make_year_month_col(depress_df, './psy_data/year_month_col/depress_ym.csv')
make_year_month_col(bipolar_df, './psy_data/year_month_col/bipolar_ym.csv')
make_year_month_col(panic_df, './psy_data/year_month_col/panic_ym.csv')

make_year_month_col(covid_df, './psy_data/year_month_col/covid_ym.csv')
make_year_month_col(relationship_df, './psy_data/year_month_col/relationship_ym.csv')
make_year_month_col(teaching_df, './psy_data/year_month_col/teaching_ym.csv')